In [1]:
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt
from tqdm import tqdm

import LinearModel as lm

import seaborn as sns
sns.set_style('white') 
sns.set_context('talk')

import pandas as pd

from sklearn.model_selection import train_test_split

SyntaxError: invalid syntax (LinearModel.py, line 121)

In [2]:
X_train_pd = pd.read_csv("./data/learn_X.csv", sep=';')
y_train_pd = pd.read_csv("./data/learn_y.csv", sep=';')
X_test_pd = pd.read_csv("./data/test_X.csv", sep=';')

y_train = np.reshape(y_train_pd.values, [-1])
X_train = X_train_pd.values
X_test = X_test_pd.values

In [3]:
np.pi

3.141592653589793

In [4]:
X_train_pd 

,Distance from the shop to the nearest subway station,District population,# of people already owning XPhone in this district,Shop's sales assistant haircolor code,Average salary in the district. RUB,Average salary in the city. RUB,# of office centers within 1 km from the shop,Number of competing shops within 1 km from the shop
0,1.132257,5195,67,1,34397,37836.861756,4,7
1,1.860470,6550,416,1,34211,16036.703314,1,19
2,2.852519,5200,683,2,19029,16752.192999,0,17
3,1.878854,1532,37,0,33736,43124.432999,4,2
4,1.356700,9719,2217,1,39449,51204.282187,0,10
5,1.118796,7724,1004,0,51926,39229.752049,2,15
6,1.877867,1116,273,1,37098,36939.416605,1,9
7,0.301564,8497,1897,2,25365,40318.444827,3,3
8,1.138741,5773,1093,1,43211,56010.341837,2,8
9,3.287832,3823,304,0,57849,66171.931064,4,8


In [4]:
def preprocess(X, means, stds):
    X[:,2] = X[:,2]/X[:,1]
    X[:,4] = X[:,4]/X[:,5]
#     X = (X - means)/stds
    X = np.hstack([X[:,:1], X[:,1+1:]])
    X = np.hstack([X[:,:2], X[:,2+1:]])
    X = np.hstack([X[:,:4], X[:,4+1:]])
    return X

In [5]:
print(X_train.shape, y_train.shape, X_test.shape)

(5000, 8) (5000,) (10000, 8)


In [6]:
means = X_train.mean(axis = 0)
stds = X_train.std(axis = 0)

In [7]:
print(means)
print(stds)

[1.76071985e+00 5.47651820e+03 8.17884800e+02 9.89000000e-01
 3.76453014e+04 4.14922977e+04 1.99160000e+00 9.62540000e+00]
[1.01173211e+00 2.58356680e+03 6.51459110e+02 8.23698367e-01
 1.30138238e+04 1.94549569e+04 1.39868847e+00 5.75877373e+00]


In [8]:
X_train = preprocess(X_train, means, stds)
X_test = preprocess(X_test, means, stds)

In [9]:
model = lm.BayesMultiLinearModel(n_models=10, mu=0.00001)

In [10]:
model.fit(X = X_train, p = y_train, epoch=20)

100%|██████████| 20/20 [01:29<00:00,  4.30s/it]


In [11]:
n_models = model.n_models()

In [12]:
n_models

10

In [13]:
mu, B, _, gamma = model.q_distribution()

In [14]:
A, b, beta = model.parameters()

In [15]:
b

array([68535.12273021, 13816.0764913 , 96957.15826895, 18540.1305337 ,
       57834.7099603 , 64085.54274709, 55849.78744064, 44261.30202903,
       54439.80509496, 80729.83980178])

In [16]:
beta

array([7052986.01472126])

In [17]:
gamma

array([ 355.,  264.,  411.,  335.,  411.,  921., 1137.,  321.,  444.,
        401.])

In [18]:
from scipy.stats import multivariate_normal

In [19]:
x = X_train[123]

In [20]:
y_train[123]

73913.480869576

In [21]:
y_test = np.zeros(X_test.shape[0])
u_test = np.zeros(X_test.shape[0])

In [26]:
N = 100
for t in tqdm(range(X_test.shape[0])):
    x = X_test[t]
    list_of_P_1 = []
    list_of_P_2 = []
    for i in range(N):
        P = 0
        pi = np.random.dirichlet(0.00001 + 1*gamma)
        
        k_2 = np.random.choice(n_models, p = pi)
        k_1 = np.argmax(pi)
        
        w = np.random.multivariate_normal(mean = mu[k_1].reshape([-1]), cov = B[k_1], size = 1)
        m_p = b[k_1].reshape([-1]) + np.sum(w*x)
        P_1 = np.random.normal(m_p, np.sqrt(1/beta), size = 1)
        
        w = np.random.multivariate_normal(mean = mu[k_2].reshape([-1]), cov = B[k_2], size = 1)
        m_p = b[k_2].reshape([-1]) + np.sum(w*x)
        P_2 = np.random.normal(m_p, np.sqrt(1/beta), size = 1)
        
        list_of_P_1.append(P_1.sum())
        list_of_P_2.append(P_2.sum())
        
    list_of_P_1 = np.array(list_of_P_1)
    list_of_P_2 = np.array(list_of_P_2)
    y_test[t] = list_of_P_1.mean()
    u_test[t] = list_of_P_2.std()


100%|██████████| 10000/10000 [07:02<00:00, 23.67it/s]


In [23]:
list_of_P = []


N = 100

for i in range(N):
    P = 0
    pi = np.random.dirichlet(0.00001 + 1*gamma)
#     k = np.random.choice(n_models, p = pi)
    k = np.argmax(pi)
#     for k in range(n_models):
    w = np.random.multivariate_normal(mean = mu[k].reshape([-1]), cov = B[k], size = 1)
    m_p = b[k].reshape([-1]) + np.sum(w*x)
#         P += pi[k]*np.random.normal(m_p, np.sqrt(1/beta), size = 1)
    P = np.random.normal(m_p, np.sqrt(1/beta), size = 1)
    list_of_P.append(P.sum())
    
list_of_P = np.array(list_of_P)

In [24]:
list_of_P.mean()

84902.34633481018

In [25]:
list_of_P.std()

0.00040670776400784215

In [28]:
print(y_test.shape)

(10000,)


In [33]:
Ans = np.vstack([y_test, u_test]).T

In [34]:
Ans.shape

(10000, 2)

In [37]:
Answer = pd.DataFrame(Ans)

In [40]:
Answer.to_csv("./answeer.csv")